In [1]:
# Packages
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import root_mean_squared_error, r2_score

import PyGRF

In [2]:
df = pd.read_csv("../Data/modeling_tuning_grf.csv")
df.head()

,Geography,lon,lat,his_num_311_per_property,neighbor_his_num_per_property,snow_depth,percent_below_poverty,percent_civilian_unemployed,per_capita_income,percent_no_highschool,...,percent_singleunits,percent_multiunit,percent_mobile_homes,percent_owneroccupiedunit,percent_crowding,percent_group_quarters,median_year_properties_built,median_value_properties_built,percent_no_vehicle,311_index_per_property
0,360290030001,1.089662e+06,1.059713e+06,52.272727,47.686137,108.149595,0.214156,0.000000,25159,0.000000,...,0.495575,0.000000,0.0,0.548263,0.069498,0.000000,1920.0,54000.0,0.108108,21.969697
1,360290170001,1.084503e+06,1.066127e+06,33.155080,43.266604,87.781572,0.042770,0.049180,45450,0.076112,...,0.758483,0.000000,0.0,0.817987,0.000000,0.000000,1920.0,43000.0,0.263383,13.368984
2,360290008001,1.084924e+06,1.036775e+06,29.487179,30.048097,149.570705,0.053443,0.020221,30061,0.209581,...,0.645892,0.016997,0.0,0.722380,0.000000,0.000000,1920.0,121000.0,0.016997,7.371795
3,360290166001,1.078699e+06,1.054669e+06,31.111111,40.989296,109.606188,0.281509,0.217252,13172,0.301370,...,0.497642,0.000000,0.0,0.674825,0.139860,0.000000,1910.0,23000.0,0.548951,11.111111
4,360290027031,1.080299e+06,1.056300e+06,65.689150,44.427546,109.196372,0.811749,0.035191,8253,0.356618,...,0.277533,0.292217,0.0,0.202509,0.030466,0.029255,1900.0,23000.0,0.564516,15.835777


In [3]:
bandwidth, local_weight, p_value = PyGRF.search_bw_lw_ISA(df["311_index_per_property"], df[['lon', 'lat']])
bandwidth = 15
local_weight = 0.14163829762242533

In [ ]:
# Standardization function
def standarize_data(data, stats):
    return (data - stats['mean']) / stats['std']

columns_to_exclude = ['Geography', 'lon', 'lat', '311_index_per_property']
columns_using = [column for column in df.columns if column not in columns_to_exclude]

ntree_list = []
mtry_list = []
maxdepth_list = []

r2_list = []
# ave_r2_list = []
rmse_list = []

number_demo = len(columns_using)
max_features = [number_demo, int(number_demo/2), int(number_demo/3), 'sqrt', 'log2']
max_depths = [5, 10, 15, 20]

num = 0
for current_ntree in range(10, 310, 10): 
    print("current_ntree: " + str(current_ntree))
    for current_mtry in max_features:
        for current_maxdepth in max_depths:
            y_grf_predict = []
            y_true = []
            
            ten_fold = KFold(n_splits=10, shuffle=True, random_state=42)
            
            # avg_r_squared = 0
            
            for train_index, test_index in ten_fold.split(df):
                # print("TEST:", test_index)
                X_train_all, X_test_all = df.iloc[train_index], df.iloc[test_index]
                X_train, X_test = X_train_all[columns_using], X_test_all[columns_using]
                y_train, y_test = X_train_all['311_index_per_property'], X_test_all['311_index_per_property']
                xy_coord = X_train_all[['lon', 'lat']]
                coords_test = X_test_all[['lon', 'lat']]
                
                training_stat = X_train.describe().transpose()
                scaled_X_train = standarize_data(X_train, training_stat)
                scaled_X_test = standarize_data(X_test, training_stat)
                
                ###########################################################################
                grf = PyGRF.PyGRFBuilder(n_estimators=current_ntree, max_features=current_mtry, max_depth = current_maxdepth, band_width=bandwidth, train_weighted = True, predict_weighted = True, 
                                  bootstrap = True, resampled = True, random_state = 42)
                ###########################################################################
                grf.fit(scaled_X_train, y_train, xy_coord)
                predict_combined, predict_global, predict_local = grf.predict(scaled_X_test, coords_test,
                                                                                  local_weight=local_weight)
                this_y_predict = predict_combined
                y_grf_predict = y_grf_predict + this_y_predict
                y_true = y_true + y_test.tolist()
                # avg_r_squared += r2_score(y_test, this_y_predict)
                
            grf_rmse = root_mean_squared_error(y_true , y_grf_predict)
            grf_r2 = r2_score(y_true, y_grf_predict)
            # avg_r_squared = avg_r_squared/10
            
            ntree_list.append(current_ntree)
            mtry_list.append(current_mtry)
            maxdepth_list.append(current_maxdepth)
    
            r2_list.append(grf_r2)
            # ave_r2_list.append(avg_r_squared)
            rmse_list.append(grf_rmse)
            
            num = num + 1
            print(num)

In [ ]:
result = {'ntree': ntree_list, "mtry": mtry_list, "maxdepth": maxdepth_list, "r2": r2_list, "rmse": rmse_list}
df_result = pd.DataFrame(result)
df_result.to_csv("../Data/modeling_tuning_grf_result.csv", index=False)